In [278]:
import functions_ml as fml
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import hamming_loss, f1_score, jaccard_score, accuracy_score, multilabel_confusion_matrix, classification_report

Depois posso separar uma análise para problemas que não haviam no teste (30 plantas, 5 produtos, etc) (o dataset mantém o índice no y_test, basta fazer merge das colunas para filtrar plantas, produtos e dps dar drop)

# Oracle

In [279]:
# Load model
TOLERANCE = 0.05
with open(f'trained_models/oracle_{TOLERANCE}percent_balanced_calibrated.pkl', 'rb') as f:
    model = pickle.load(f)

# Test set ("new instances")

In [280]:
TEST_TOLERANCE_LIMIT = TOLERANCE / 100
test_results = pd.read_csv('datasets/test_instances_results.csv')
test_features = pd.read_csv('datasets/test_instances_features.csv')
test_set = fml.create_dataset(test_features, test_results)
test_set = fml.create_multi_label_target(test_set, TEST_TOLERANCE_LIMIT)

In [281]:
target_cols = [t for t in test_set.columns if t.startswith('RF')]
for t in target_cols:
    print(t, ':', test_set[t].sum())

RF_1_0 : 13
RF_2_0 : 16
RF_2_1 : 13
RF_3_0 : 13
RF_3_1 : 11
RF_3_2 : 11
RF_4_0 : 13
RF_4_1 : 13
RF_4_2 : 11
RF_4_3 : 9
RF_6_0 : 13
RF_6_1 : 12
RF_6_2 : 13
RF_6_3 : 12
RF_6_4 : 12
RF_6_5 : 9
RF_T_0 : 12


In [282]:
# Preprocessing (deve ser igual ao aplicado no desenvolvimento)
X_test = test_set.drop(columns=target_cols + ['instance'])
X_test = fml.binary_feature_selection(X_test)
y_test = test_set[target_cols]

## Predict

In [283]:
y_pred = pd.DataFrame(model.predict(X_test), columns=y_test.columns, index=y_test.index)
probabilities = pd.DataFrame(model.predict_proba(X_test), columns=y_test.columns, index=y_test.index)

In [284]:
hamming = hamming_loss(y_test, y_pred)
print("Hamming Loss:", hamming)

f1 = f1_score(y_test, y_pred, average='micro')
print("Micro-Averaged F1 Score:", f1)

jaccard = jaccard_score(y_test, y_pred, average='samples')
print("Jaccard Similarity Score:", jaccard)

subset_accuracy = accuracy_score(y_test, y_pred)
print("Subset Accuracy:", subset_accuracy)

Hamming Loss: 0.11764705882352941
Micro-Averaged F1 Score: 0.8716577540106952
Jaccard Similarity Score: 0.4619076797385621
Subset Accuracy: 0.25


In [285]:
print(classification_report(y_test, y_pred, target_names=y_test.columns, zero_division="warn"))

              precision    recall  f1-score   support

      RF_1_0       0.73      0.62      0.67        13
      RF_2_0       1.00      0.75      0.86        16
      RF_2_1       1.00      0.77      0.87        13
      RF_3_0       1.00      0.77      0.87        13
      RF_3_1       1.00      0.91      0.95        11
      RF_3_2       1.00      0.73      0.84        11
      RF_4_0       1.00      0.77      0.87        13
      RF_4_1       1.00      0.77      0.87        13
      RF_4_2       1.00      0.82      0.90        11
      RF_4_3       0.88      0.78      0.82         9
      RF_6_0       1.00      0.77      0.87        13
      RF_6_1       1.00      0.83      0.91        12
      RF_6_2       1.00      0.77      0.87        13
      RF_6_3       1.00      0.83      0.91        12
      RF_6_4       1.00      0.83      0.91        12
      RF_6_5       0.89      0.89      0.89         9
      RF_T_0       1.00      0.92      0.96        12

   micro avg       0.97   

/home/lucasmalheiros/Documents/Pesquisa Operacional/tcc-relax-and-fix/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [286]:
method_evaluation_results = []
for method in y_test.columns:
    # Calculate true positives (correct assignments)
    true_positives = ((y_pred == 1) & (y_test == 1))[method].sum()

    # Calculate false positives (predicted as 1 but actual is 0)
    false_positives = ((y_pred == 1) & (y_test == 0))[method].sum()

    # Calculate false negatives (predicted as 0 but actual is 1)
    false_negatives = ((y_pred == 0) & (y_test == 1))[method].sum()

    # Calculate Precision
    if (true_positives + false_positives) > 0:
        precision = true_positives / (true_positives + false_positives)
    else:
        precision = 0

    # Calculate Recall
    if (true_positives + false_negatives) > 0:
        recall = true_positives / (true_positives + false_negatives)
    else:
        recall = 0 

    # Calculate F1-Score
    if (precision + recall) > 0:
        f1_score = (2 * precision * recall) / (precision + recall)
    else:
        f1_score = 0

    # Append results to the list
    method_evaluation_results.append({
        'Method': method,
        'True Positives': true_positives,
        'False Positives': false_positives,
        'False Negatives': false_negatives,
        'Precision (%)': f'{precision * 100 :.2f}',
        'Recall (%)': f'{recall * 100 :.2f}',
        'F1-Score': f'{f1_score * 100 :.2f}'
    })
method_evaluation_results_df = pd.DataFrame(method_evaluation_results)

In [287]:
method_evaluation_results_df

,Method,True Positives,False Positives,False Negatives,Precision (%),Recall (%),F1-Score
0,RF_1_0,8,3,5,72.73,61.54,66.67
1,RF_2_0,12,0,4,100.00,75.00,85.71
2,RF_2_1,10,0,3,100.00,76.92,86.96
3,RF_3_0,10,0,3,100.00,76.92,86.96
4,RF_3_1,10,0,1,100.00,90.91,95.24
5,RF_3_2,8,0,3,100.00,72.73,84.21
6,RF_4_0,10,0,3,100.00,76.92,86.96
7,RF_4_1,10,0,3,100.00,76.92,86.96
8,RF_4_2,9,0,2,100.00,81.82,90.00
9,RF_4_3,7,1,2,87.50,77.78,82.35


## Model probabilities

### Select top K

In [288]:
ranked_methods = np.argsort(-probabilities, axis=1)
# Top-K Selection: Select top-K methods based on ranking
TOP_K = 3
top_k_methods = np.zeros_like(probabilities, dtype=int)

# Set top-K methods as selected (binary)
for i, row in enumerate(ranked_methods):
    top_indices = row[:TOP_K]  # Get top-K indices for this instance
    top_k_methods[i, top_indices] = 1

# Convert to DataFrame for comparison
top_k_methods_df = pd.DataFrame(top_k_methods, columns=y_test.columns, index=y_test.index)

# Display the top-K selected methods
top_k_methods_df.head()

,RF_1_0,RF_2_0,RF_2_1,RF_3_0,RF_3_1,RF_3_2,RF_4_0,RF_4_1,RF_4_2,RF_4_3,RF_6_0,RF_6_1,RF_6_2,RF_6_3,RF_6_4,RF_6_5,RF_T_0
2,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1
4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0
23,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0


In [289]:
len(test_set)

24

In [290]:
# Element-wise multiplication to find correct predictions
correct_predictions = top_k_methods_df * test_set

# Sum the total number of correct predictions
total_correct = correct_predictions.sum().sum()  # Sum across all rows and columns
print(f"Total Correct Predictions: {total_correct}")

# Optionally, calculate the number of correct predictions per instance
correct_per_instance = correct_predictions.sum(axis=1)
print("\nCorrect Predictions Per Instance:")
print(correct_per_instance)

# Optionally, calculate the accuracy as a percentage
total_possible = TOP_K * len(top_k_methods_df)  # Total number of actual positive labels
accuracy = (total_correct / total_possible) * 100
print(f"\nAccuracy: {accuracy:.2f}%")

# Precision
if TOP_K == 1:
    precision = total_correct / len(y_test) * 100
    print(f"Precision: {precision:.2f}")

Total Correct Predictions: 46.0

Correct Predictions Per Instance:
2     3.0
0     3.0
4     1.0
13    3.0
23    3.0
21    1.0
7     3.0
16    3.0
18    3.0
17    3.0
14    0.0
12    3.0
10    3.0
8     0.0
3     1.0
15    1.0
20    0.0
19    1.0
22    0.0
5     2.0
1     2.0
6     3.0
11    1.0
9     3.0
dtype: float64

Accuracy: 63.89%


In [291]:
# Step 1: Calculate the number of correct predictions per row (instance)
correct_per_instance = correct_predictions.sum(axis=1)

# Step 2: Identify rows with zero correct predictions
missed_all = (correct_per_instance == 0).sum()  # Count rows with no correct predictions
total_instances = len(correct_predictions)  # Total number of rows

# Step 3: Calculate the accuracy for missed rows
missed_accuracy = (missed_all / total_instances) * 100

# Output the results
print(f"Number of Rows with All Incorrect Predictions: {missed_all}")
print(f"Percentage of Rows with All Incorrect Predictions: {missed_accuracy:.2f}%")

Number of Rows with All Incorrect Predictions: 4
Percentage of Rows with All Incorrect Predictions: 16.67%


### Performance what if only 1 method

In [292]:
# Step 1: Initialize a DataFrame to store results
method_evaluation_results = []

# Step 2: Loop through each method
for method in y_test.columns:
    # Create a binary matrix where the current method is always 1 and others are 0
    single_method_matrix = np.zeros_like(y_test, dtype=int)
    single_method_matrix[:, y_test.columns.get_loc(method)] = 1  # Set current method column to 1
    
    # Convert to DataFrame
    single_method_df = pd.DataFrame(single_method_matrix, columns=y_test.columns, index=y_test.index)

    # Calculate true positives (correct assignments)
    true_positives = ((single_method_df == 1) & (y_test == 1))[method].sum()

    # Calculate false positives (predicted as 1 but actual is 0)
    false_positives = ((single_method_df == 1) & (y_test == 0))[method].sum()

    # Calculate false negatives (predicted as 0 but actual is 1)
    false_negatives = ((single_method_df == 0) & (y_test == 1))[method].sum()

    # Calculate Precision
    if (true_positives + false_positives) > 0:
        precision = true_positives / (true_positives + false_positives)
    else:
        precision = 0.0  # Handle division by zero

    # Calculate Recall
    if (true_positives + false_negatives) > 0:
        recall = true_positives / (true_positives + false_negatives)
    else:
        recall = 0.0  # Handle division by zero

    # Append results to the list
    method_evaluation_results.append({
        'Method': method,
        'True Positives': true_positives,
        'False Positives': false_positives,
        'False Negatives': false_negatives,
        'Precision (%)': precision * 100,
        'Recall (%)': recall * 100
    })

In [293]:
# Step 3: Convert results to a DataFrame for easier visualization
method_evaluation_results_df = pd.DataFrame(method_evaluation_results)

# Display the evaluation results
method_evaluation_results_df

,Method,True Positives,False Positives,False Negatives,Precision (%),Recall (%)
0,RF_1_0,13,11,0,54.166667,100.0
1,RF_2_0,16,8,0,66.666667,100.0
2,RF_2_1,13,11,0,54.166667,100.0
3,RF_3_0,13,11,0,54.166667,100.0
4,RF_3_1,11,13,0,45.833333,100.0
5,RF_3_2,11,13,0,45.833333,100.0
6,RF_4_0,13,11,0,54.166667,100.0
7,RF_4_1,13,11,0,54.166667,100.0
8,RF_4_2,11,13,0,45.833333,100.0
9,RF_4_3,9,15,0,37.500000,100.0
